In [52]:
import pickle
import keras
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Dense, LSTM, Dropout, Input
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [44]:
epochs = 75
batch_size = 32
lr = 1e-3
train_size = 900
n_timesteps = 60

In [45]:
dataset = pd.read_csv('TSLA-2.csv')
dataset

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-06-11,21.500668,22.310667,21.500000,22.139999,22.139999,197752500
1,2018-06-12,22.980000,23.664667,22.533333,22.851334,22.851334,335211000
2,2018-06-13,23.114000,23.146667,22.653334,22.985332,22.985332,142047000
3,2018-06-14,23.175333,23.916668,23.106667,23.848000,23.848000,164715000
4,2018-06-15,23.589333,24.311333,23.416668,23.878000,23.878000,162724500
...,...,...,...,...,...,...,...
1253,2023-06-02,210.149994,217.250000,209.750000,213.970001,213.970001,164129000
1254,2023-06-05,217.800003,221.289993,214.520004,217.610001,217.610001,151143100
1255,2023-06-06,216.139999,221.910004,212.529999,221.309998,221.309998,146911600
1256,2023-06-07,228.000000,230.830002,223.199997,224.570007,224.570007,185710800


In [46]:
train_set = dataset.iloc[:train_size, 1:2].values
test_set = dataset.iloc[train_size: 1:2].values

In [47]:
scaler = MinMaxScaler(feature_range = (0, 1))
train_set_scaled = scaler.fit_transform(train_set)

X_train = []
y_train = []

for i in range(n_timesteps, train_size):
    X_train.append(train_set_scaled[i-n_timesteps:i, 0])
    y_train.append(train_set_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [48]:
def build_model():
    model_input = Input(shape=X_train.shape[1:])

    lstm1 = LSTM(units=64, 
                 return_sequences=True,
                 dropout=0.2)(model_input)

    lstm2 = LSTM(units=64, 
                 return_sequences=True,
                 dropout=0.2)(lstm1)
    
    lstm3 = LSTM(units=64,
                 dropout=0.2)(lstm2)

    output = Dense(units=1)(lstm3)

    model = Model(model_input, output)
    model.compile(optimizer = Adam(lr), loss = 'mse')
    
    return model

model = build_model()
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 60, 1)]           0         
                                                                 
 lstm_14 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_15 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_16 (LSTM)              (None, 64)                33024     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 83,009
Trainable params: 83,009
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(X_train, y_train,
         epochs=epochs,
         batch_size=batch_size)

Epoch 1/75
27/27 [==============================] - 5s 14ms/step - loss: 0.0280
Epoch 2/75
27/27 [==============================] - 0s 10ms/step - loss: 0.0037
Epoch 3/75
27/27 [==============================] - 0s 10ms/step - loss: 0.0037
Epoch 4/75
27/27 [==============================] - 0s 11ms/step - loss: 0.0039
Epoch 5/75
27/27 [==============================] - 0s 11ms/step - loss: 0.0032
Epoch 6/75
27/27 [==============================] - 0s 10ms/step - loss: 0.0043
Epoch 7/75
27/27 [==============================] - 0s 10ms/step - loss: 0.0033
Epoch 8/75
27/27 [==============================] - 0s 11ms/step - loss: 0.0035
Epoch 9/75
27/27 [==============================] - 0s 10ms/step - loss: 0.0030
Epoch 10/75
27/27 [==============================] - 0s 10ms/step - loss: 0.0034
Epoch 11/75
27/27 [==============================] - 0s 11ms/step - loss: 0.0039
Epoch 12/75
27/27 [==============================] - 0s 10ms/step - loss: 0.0034
Epoch 13/75
27/27 [==================

In [50]:
# serialize lstm model
model.save('lstm.h5')

# serialize minmax scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)